In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
import src.cleaning as clean
import numpy as np
import seaborn as sns
import src.analysis.utils as utils

In [ ]:
sales_data = pd.read_csv('../../data/anon_data.csv')
cleaned_data = clean.clean_data(sales_data)

In [ ]:
# Assume order date is the earliest shipping date
cleaned_data['order_date'] = cleaned_data.groupby('sales_order_number')['ship_date'].transform(min)

In [ ]:
#Restrict to only orders being sent not returned - being quantity > 0 
positive_quantity = cleaned_data.loc[cleaned_data['quantity']>0, :]

# Most Frequent Customers

In [ ]:
# Number of order days by customer, restriction being quantity > 0 
customer_order_dates = positive_quantity.groupby('customer_id')['order_date'].unique()
customer_regularity = customer_order_dates.apply(len).sort_values(ascending=False)
top_10_regular_customers = customer_regularity.iloc[list(range(0, 10))]
top_10_regular_customers

In [ ]:
top_10_regular_customers.sum()

In [ ]:
customer_regularity[customer_regularity>customer_regularity.quantile(0.9)]

In [ ]:
average_days_between_customer_orders = customer_order_dates.loc[top_10_regular_customers.index].apply(utils.calculate_days_between)
average_days_between_customer_orders.apply(np.mean)

# Regularly Ordered by Regulars

In [ ]:
positive_quantity_by_regulars = positive_quantity.loc[positive_quantity['customer_id'].isin(top_10_regular_customers.index), :]
positive_quantity_by_regulars.groupby('customer_id')['order_date'].unique().apply(len).sort_values(ascending=False).sum()

In [ ]:
# Groupby brand and item
dates_of_items_ordered = positive_quantity_by_regulars.groupby(['brand', 'product_type'])['order_date'].unique()
number_of_days_item_ordered = dates_of_items_ordered.apply(len).sort_values(ascending=False)
most_commonly_ordered_items = number_of_days_item_ordered.loc[number_of_days_item_ordered.iloc[list(range(10))].index]
most_commonly_ordered_items

In [ ]:
most_commonly_ordered_items.sum()

In [ ]:
days_between_items_ordered = dates_of_items_ordered[most_commonly_ordered_items.index].apply(utils.calculate_days_between).apply(np.mean).sort_values(ascending=True)
days_between_items_ordered